In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [11]:
# 处理数据，uv处理成风向和风速
data = pd.read_csv('./used_multi.csv')
wnd_dir = []
wnd_spd = []
for i in range(len(data)):
    u = data['u'][i]
    v = data['v'][i]
    fx = 999.9
    if (u>0) & (v>0):
        fx = fx = 270 - math.atan(v / u) * 180 / math.pi
    elif (u < 0) & (v > 0):
        fx = 90 - math.atan(v / u) * 180 / math.pi
    elif (u < 0) & (v < 0):
        fx = 90 - math.atan(v / u) * 180 / math.pi
    elif (u > 0) & (v < 0):
        fx = 270 - math.atan(v / u) * 180 / math.pi
    elif (u == 0) & (v > 0):
        fx = 180
    elif (u == 0) & (v < 0):
        fx = 0
    elif (u > 0) & (v == 0):
        fx = 270
    elif (u < 0) & (v == 0):
        fx = 90
    elif (u == 0) & (v == 0):
        fx = 999.9
    else:
        fx = 999.9
    wnd_dir.append(fx)

    # 风速是uv分量的平方和
    fs = math.sqrt(math.pow(u,2) + math.pow(v,2))
    wnd_spd.append(fs)
# 增加到文件中
add_file_path = './used_multi.csv'
data['wnd_dir'] = wnd_dir
data['wnd_spd'] = wnd_spd
data.to_csv(add_file_path,header=True,index=None)